<font face="Calibri" size="2"> <i>Open SAR Toolkit - Tutorial 1, version 1.1,  November 2019. Andreas Vollrath, ESA/ESRIN phi-lab</i>
</font>

![title](auxiliary/header_image.PNG)

--------

# OST Tutorial I 
## Pre-processing your first Sentinel-1 image with OST

--------

**Short description**

This notebook introduces you to the *Sentinel_1Scene* class of the Open SAR Toolkit and demonstrates how it can be used to download, extract metadata and pre-process a single Sentinel-1 scene.

--------

**Requirements**

- a PC/Mac with at least 16GB of RAM
- about 4 GB of free disk space
- a NASA Earthdata account with signed EULA for use of https://search.asf.alaska.edu (just register directly there)
--------

**NOTE:** all cells that have an * after its number can be executed without changing any code. 

### 1\* - Import the OST *Sentinel1_Scene* class

In [ ]:
# these imports we need to handle the folders, independent of the OS
import os
from os.path import join
from pathlib import Path
from pprint import pprint

# this is the s1Scene class, that basically handles all the workflow from beginning to the end
from ost import Sentinel1_Scene

### 2* - Create a folder for our outputs

By executing this cell, a new folder will be created and the path will be written to the *processing_dir* variable

In [ ]:
# get home folder
home = str(Path.home())

# create a processing directory
output_dir = join(home, 'OpenSarToolkit', 'Tutorial_1')
os.makedirs(output_dir, exist_ok=True)

### 3* - Choose scene ID and display some metadata

In order to initialize an instance of the *Sentinel1_Scene* class, all we need is a valid scene id of a Sentinel-1 product. 

In [ ]:
# create a S1Scene class instance based on the scene identifier of the first ever Dual-Pol Sentinel-1 IW product
#scene_id = 'S1A_IW_GRDH_1SDV_20141003T040550_20141003T040619_002660_002F64_EC04' # actually only available from ASF's data archive
scene_id = 'S1A_IW_GRDH_1SDV_20191116T170638_20191116T170703_029939_036AAB_070F'
# create an S1Scene instance
s1 = Sentinel1_Scene(scene_id)

# print summarising infos about the scene
s1.info()

### 4* Download scene

The first step is to download the selected scene. In our case we chose the first regular Sentinel-1 IW product acquired in the dual-polarisation VV/VH acquired the 3rd of October 2014. 
OST provides download from 3 different mirrors, ESA's scihub, CNES PEPS and Alaska Satellite Facility (NASA Earthdata). Since ESA's official scihub became a rolling archive, and so is PEPS, best is to download from the fantastic **Alaska Satellite Facility** mirror (selection 2), which holds the full Sentinel-1 archive online. 

**Note I:** You can interrupt the download and restart it. The download will continue from where it stopped.

**Note II:** At the end the downloaded file will be checked. This assures that the download went fine and we can use it for processing. In addiiton, OST will also magically remember that this file has been successfully downloaded.

In [ ]:
s1.download(output_dir)

### 5* - Create an ARD product

Analysis Ready Data is interpreted differently by different persons. 

The default ARD type is *'OST Standard'*. For GRD products you can select other pre-defined ARD types, or it allows customising single ARD parameters.

See in the cell below the different definitions possible, and also note how the resolution and the resampling of the image during terrain correction is changed at the bottom. In this way, all parameters can be customised. 

In [ ]:
print(' Our ARD parameters dictionary contains 3 keys.')
pprint(s1.ard_parameters.keys())
print('')
print(' Dictionary of our standard OST ARD parameters for the scene processing:')
pprint(s1.ard_parameters['single ARD'])
print('')


# we change ARD type
# possible coices are:
# 'OST Standard', 'OST Flat', 'CEOS', 'Earth Engine'
s1.get_ard_parameters('Earth Engine')
print(' Dictionary of Earth Engine ARD parameters for the scene processing:')
pprint(s1.ard_parameters['single ARD'])
print('')

# we cusomize the resolution and image resampling
s1.ard_parameters['single ARD']['resolution'] = 50
s1.ard_parameters['single ARD']['dem']['image resampling'] = 'BILINEAR_INTERPOLATION'
pprint(s1.ard_parameters['single ARD'])

Our *Sentinel1_Scene* class comes with the build-in method *create_ard* to produce a standardised ARD product based on the ARD dictionary above. 

To run the command we just have to provide: 
- the path to the downloaded file. We can use the *get_path* method in conjunction with the download directory provided
- a directory where the outputfiles will be written to
- a filename prefix (the output format will be the standard SNAP Dimap format, consisting of the dim-file and the data directory)
- and a directory for storing temporary files (can not be the same as the output folder)

In [ ]:
s1.create_ard(infile = s1.get_path(output_dir),
              out_dir = output_dir,   
              out_prefix = s1.start_date,   
              temp_dir = '/tmp') # needs to be different from our output dir

print(' The path to our newly created ARD product can be obtained the following way:')
s1.ard_dimap

### 6* - Create a RGB color composite

Sentinel-1 scenes usually consist of two polarisation bands. In order to create a 3 channel RGB composite a ratio between the Co- (VV or HH) and the Cross-polarised (VH or HV) band is added. The *create_rgb* method takes the *ard_dimap* file and converts it to a 3-channel GeoTiff.

In [ ]:
s1.create_rgb(outfile = join(output_dir, '{}.tif'.format(s1.start_date)))

print(' The path to our newly created RGB product can be obtained the follwing way:')
s1.ard_rgb

### 7* - Visualise the RGB composite

We can plot the newly created RGB image with the *visualise_rgb* method. A *shrink_factor* is added, which reduces resolution in favour of memory requirements for plotting.

In [ ]:
#---------------------------------------------------
# for plotting purposes we use this iPython magic
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (23, 23)
#---------------------------------------------------
s1.visualise_rgb(shrink_factor=15)

### 8* - Create thumbnail image

For some it might be interesting to create a small thumbnail image in Jpeg format. The *create_rgb_thumbnail* method allows for this. 

In [ ]:
# define a filename for our thumbnail image
path_to_thumbnail = join(output_dir, '{}.TN.jpg'.format(s1.start_date))
# create the thumbnail image
s1.create_rgb_thumbnail(outfile = path_to_thumbnail)

In [ ]:
import imageio
img = imageio.imread(path_to_thumbnail)
!ls -sh {path_to_thumbnail}
plt.imshow(img)